In [ ]:
from imutils import face_utils
import dlib
import cv2
import os
from scipy.spatial import distance
import numpy as np
from math import sqrt
import tensorflow as tf
from google.colab import drive

In [ ]:
def get_landmarks_gt(path):
    landmarks_gt = list()
    f = open(path, "r")
    lines = f.readlines()
    for i, line in enumerate(lines):
        if i < 3 or i == len(lines)-1: continue
        x_gt, y_gt = line.split()
        landmarks_gt.append(np.array([float(x_gt), float(y_gt)]))
    return np.array(landmarks_gt)

In [ ]:
def get_annotated_face(rects, landmarks_gt):
    rect_dict = dict()
    for i, face in enumerate(rects):
        c = 0
        x = face.rect.left()
        y = face.rect.top()
        w = face.rect.right() - x
        h = face.rect.bottom() - y
        for (x_gt, y_gt) in landmarks_gt:
            if (x <= x_gt <= x + w or x >= x_gt >= x + w) and (y <= y_gt <= y + h or y >= y_gt >= y + h):
                c += 1
        rect_dict[i] = c
    max_key = max(rect_dict, key=rect_dict.get)
    if rect_dict[max_key]/68 < 0.5:
      return rects[max_key].rect, True
    return rects[max_key].rect, False

In [ ]:
def get_iod(landmarks):
    # Left eye center
    x_left = [p[0] for p in landmarks[36:42]]
    y_left = [p[1] for p in landmarks[36:42]]
    centroid_left = (int(sum(x_left) / len(landmarks[36:42])), int(sum(y_left) / len(landmarks[36:42])))
    # Right eye center
    x_right = [p[0] for p in landmarks[42:48]]
    y_right = [p[1] for p in landmarks[42:48]]
    centroid_right = (int(sum(x_right) / len(landmarks[42:48])), int(sum(y_right) / len(landmarks[42:48])))
    # IOD
    return distance.euclidean(centroid_left, centroid_right)

In [ ]:
def convert_and_trim_bb(image, rect):
    startX = rect.left()
    startY = rect.top()
    endX = rect.right()
    endY = rect.bottom()
    startX = max(0, startX)
    startY = max(0, startY)
    endX = min(endX, image.shape[1])
    endY = min(endY, image.shape[0])
    w = endX - startX
    h = endY - startY
    return (startX, startY, w, h)

In [ ]:
detector = dlib.cnn_face_detection_model_v1("./drive/MyDrive/mmod_human_face_detector.dat")
predictor = dlib.shape_predictor("./drive/MyDrive/shape_predictor_68_face_landmarks.dat")

In [ ]:
input_dir = "/content/drive/MyDrive/SB_datasets/ibug/ibug_masked/"
output_dir = "./data/"

In [ ]:
nrmse_global = list()
count_incorrect = 0
count_all = 0

for count, filename in enumerate(os.listdir(input_dir)):
    if not (filename.__contains__(".jpg") or filename.__contains__(".png")): continue
    error = False
    count_all += 1
    print("Processing: %s (%d)" % (filename, count_all))

    filename_s = os.path.splitext(filename)[0]
    landmarks_gt = get_landmarks_gt(input_dir+filename_s+".pts")

    image = cv2.imread(input_dir+filename)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Draw all ground truth landmarks
    for (x, y) in landmarks_gt:
        cv2.circle(image, (int(x), int(y)), 2, (0, 0, 255), -1)

    # Get faces
    rects = detector(rgb, 0)
    for face in rects:
      x = face.rect.left()
      y = face.rect.top()
      w = face.rect.right() - x
      h = face.rect.bottom() - y
      cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 0), 2)
    if len(rects) == 0:
      cv2.imwrite(output_dir+"errors/"+filename, image)
      count_incorrect += 1
      continue
    else:
      face, error = get_annotated_face(rects, landmarks_gt)

    # Make the prediction and transform it to numpy array
    landmarks = predictor(rgb, face)
    landmarks = face_utils.shape_to_np(landmarks)

    # IOD
    iod = get_iod(landmarks)

    distances = list()
    # Draw all predicted landmarks
    for i, (x, y) in enumerate(landmarks):
        cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        distance_normalized = (distance.euclidean(landmarks[i], landmarks_gt[i]) / iod)**2
        distances.append(distance_normalized)
    nrmse_local = sqrt(sum(distances)/len(distances))
    if not error:
      nrmse_global.append(nrmse_local)

    if error:
      count_incorrect += 1
      cv2.imwrite(output_dir+"errors/"+filename, image)
    else:
      cv2.imwrite(output_dir + filename, image)

print("NRMSE: %.5f, skipped %d of %d (%.3f)" % (np.mean(np.array(nrmse_global)), count_incorrect, count_all, count_incorrect/count_all))

Processing: image_004_1.jpg (1)
Processing: image_003_1.jpg (2)
Processing: image_005_1.jpg (3)
Processing: image_008_1.jpg (4)
Processing: image_010_1.jpg (5)
Processing: image_011_1.jpg (6)
Processing: image_013.jpg (7)
Processing: image_012.jpg (8)
Processing: image_014.jpg (9)
Processing: image_014_01.jpg (10)
Processing: image_014_02.jpg (11)
Processing: image_015_1.jpg (12)
Processing: image_017_1.jpg (13)
Processing: image_016_1.jpg (14)
Processing: image_019_1.jpg (15)
Processing: image_021_1.jpg (16)
Processing: image_020_1.jpg (17)
Processing: image_022.jpg (18)
Processing: image_023_1.jpg (19)
Processing: image_024_1.jpg (20)
Processing: image_026_1.jpg (21)
Processing: image_025_1.jpg (22)
Processing: image_029_1.jpg (23)
Processing: image_033_1.jpg (24)
Processing: image_034_1.jpg (25)
Processing: image_035.jpg (26)
Processing: image_041_1.jpg (27)
Processing: image_040.jpg (28)
Processing: image_042.jpg (29)
Processing: image_043_1.jpg (30)
Processing: image_045_1.jpg (31

In [ ]:
!zip -r /content/file.zip ./data/


  adding: data/ (stored 0%)
  adding: data/image_040.jpg (deflated 5%)
  adding: data/image_115_01.jpg (deflated 3%)
  adding: data/image_092.jpg (deflated 0%)
  adding: data/image_010_1.jpg (deflated 5%)
  adding: data/image_028.jpg (deflated 5%)
  adding: data/image_118_02.jpg (deflated 2%)
  adding: data/image_060_1.jpg (deflated 5%)
  adding: data/image_045_1.jpg (deflated 3%)
  adding: data/image_005_1.jpg (deflated 2%)
  adding: data/image_105.jpg (deflated 2%)
  adding: data/image_092 _01.jpg (deflated 0%)
  adding: data/image_083_2.jpg (deflated 4%)
  adding: data/image_100.jpg (deflated 4%)
  adding: data/image_015_1.jpg (deflated 9%)
  adding: data/image_004_1.jpg (deflated 2%)
  adding: data/image_109.jpg (deflated 4%)
  adding: data/image_055.jpg (deflated 3%)
  adding: data/image_066_1.jpg (deflated 4%)
  adding: data/image_074.jpg (deflated 0%)
  adding: data/image_110_01.jpg (deflated 3%)
  adding: data/image_108.jpg (deflated 4%)
  adding: data/image_091.jpg (deflated 3

In [ ]:
rm -rf ./data/*

In [ ]:
mkdir ./data/errors

In [ ]:
rm -rf file.zip